<a href="https://colab.research.google.com/github/martindevoto/machine-learning-notebooks-personal/blob/main/Gradio_Translation_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio git+https://github.com/huggingface/transformers gradio torch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import gradio as gr
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [3]:
tokenizer = AutoTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

model = \
    AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')

In [4]:
device = 0 if torch.cuda.is_available() else -1

In [5]:
LANGS = ["ace_Arab", "eng_Latn", "fra_Latn", "spa_Latn"]
LANGS_DISPLAY_NAME = ['Arab', 'English', 'French', 'Spanish']

In [6]:
langs_mapper = dict(zip(LANGS_DISPLAY_NAME, LANGS))
langs_mapper

{'Arab': 'ace_Arab',
 'English': 'eng_Latn',
 'French': 'fra_Latn',
 'Spanish': 'spa_Latn'}

In [7]:
def translate(text, src_lang="spa_Latn", tgt_lang="eng_Latn"):
    """Translate the text from source language to target language."""
    translation_pipeline = pipeline('translation',
                                    tokenizer=tokenizer,
                                    model=model,
                                    src_lang=src_lang,
                                    tgt_lang=tgt_lang,
                                    max_length=400, 
                                    device=device)
    result = translation_pipeline(text)
    return result[0]['translation_text']

In [8]:
demo = gr.Interface(
    fn=translate,
    inputs=[
        gr.components.Textbox(label='Text'),
        gr.components.Dropdown(label='Source Language', choices=LANGS),
        gr.components.Dropdown(label='Target Language', choices=LANGS),
    ],
    outputs=['text'],
    examples=[["Building a translation demo with Gradio is so easy!", "eng_Latn", "spa_Latn"]],
    # examples=[["Building a translation demo with Gradio is so easy!", "English", "Spanish"]],
    cache_examples=False,
    title='Translation Demo Page',
    # description="This demo is a simplified version of the original [NLLB-Translator](https://huggingface.co/spaces/Narrativaai/NLLB-Translator) space"
    description="This is a translation demo based on the [NLLB-Translator](https://huggingface.co/spaces/Narrativaai/NLLB-Translator) space"
)

In [9]:
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # Demo de servicio traducción de español a inglés
        Escribí el texto que quieras traducir de español a inglés
        """
    )
    inp = gr.Textbox(label='Texto en español', placeholder='Escribí acá')
    out = gr.Textbox(label='Texto traducido al inglés')
    inp.change(translate, inp, out)


In [10]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://75c04bfa1147d7c9cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
